In [1]:
import warnings

warnings.filterwarnings('ignore')

import re
import json
import pandas as pd

from selenium import webdriver  # 导入库
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from fake_useragent import UserAgent
import pandas as pd
import time
import random

import requests
from lxml import etree

In [2]:
pd.set_option('max_row', 300)

In [3]:
def ini_browser():
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized")
    #     options.add_argument('User-Agent={}'.format(UserAgent().random))
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    #    prefs = {"profile.managed_default_content_settings.images": 2}
    #    options.add_experimental_option("prefs", prefs)
    browser = webdriver.Chrome(chrome_options=options)
    browser.get('http://www.baidu.com')
    return browser

In [4]:
browser1 = ini_browser()
browser2 = ini_browser()

In [5]:
# base_url = 'http://data.bank.hexun.com/lccp/alllccp.aspx'
# browser1.get(base_url)

In [6]:
# i = 0
# while i<2:
#     browser1.execute_script("window.scrollTo(0, document.body.scrollHeight)")
#     time.sleep(1)
#     i+=1
# time.sleep(5)

In [7]:
# tree1 = etree.HTML(browser1.page_source)  # 获取源码

In [8]:
p=1
all_info = []

while p<37:
    base_url = 'http://data.bank.hexun.com/lccp/AllLccp.aspx?col=fld_issenddate&tag=desc&orderMarks=&page={p}'.format(p=p)
    browser1.get(base_url)
    tree1 = etree.HTML(browser1.page_source) 
    
    t = 0
    while t<2:
        browser1.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(1)
        t+=1
    time.sleep(5)
    
    prod_baselink = 'http://data.bank.hexun.com/lccp/'
    i = 2

    while i <= 21:
        prod_name = tree1.xpath("//div[@class='maincont']/div[@class='mainbox']/div[@class='mbox']/div[@class='mark']/table[@id='Table1']/tbody/tr[{i}]/td[@class='dr']/b/a".format(i=i))[0].text
        prod_sublink = tree1.xpath("//div[@class='maincont']/div[@class='mainbox']/div[@class='mbox']/div[@class='mark']/table[@id='Table1']/tbody/tr[{i}]/td[@class='dr']/b/a//@href".format(i=i))[0]
        prod_link = prod_baselink + prod_sublink
        browser2.get(prod_link)
        tree2 = etree.HTML(browser2.page_source)

        prod_info = {}

        time.sleep(2)
        
        prod_name = tree2.xpath("//div[@class='maincont']/div[@class='mainbox']/div[@class='mbox']/div[@class='dzMain']/div[@class='dzMain'][1]/div[@class='Area443 fl']/div[@class='presell']/div[@class='fl prestxt']")[0].text
        prod_info['产品名称'] = prod_name
        print(prod_name)
        
        prod_img = tree2.xpath("//div[@class='maincont']/div[@class='mainbox']/div[@class='mbox']/div[@class='dzMain']/div[@class='dzMain'][1]/div[@class='Area443 fl']/div[@class='presell']/div[@class='logo']/a/img/@src")[0]
        prod_info['图片地址'] = prod_img
        
        row = 1
        while row<=8:
            key1 = tree2.xpath("//div[@class='mainbox']/div[@class='mbox']/div[@class='dzMain']/div[@class='dzMain'][1]/div[@class='Area443 fl']/div[2]/table[@class='dzTable']/tbody/tr[{row}]/th[1]".format(row=row))[0].text.replace('：', '')
            val1 = tree2.xpath("//div[@class='mainbox']/div[@class='mbox']/div[@class='dzMain']/div[@class='dzMain'][1]/div[@class='Area443 fl']/div[2]/table[@class='dzTable']/tbody/tr[{row}]/td[1]".format(row=row))[0].text.replace('：', '')
            key2 = tree2.xpath("//div[@class='mainbox']/div[@class='mbox']/div[@class='dzMain']/div[@class='dzMain'][1]/div[@class='Area443 fl']/div[2]/table[@class='dzTable']/tbody/tr[{row}]/th[2]".format(row=row))[0].text.replace('：', '')
            val2 = tree2.xpath("//div[@class='mainbox']/div[@class='mbox']/div[@class='dzMain']/div[@class='dzMain'][1]/div[@class='Area443 fl']/div[2]/table[@class='dzTable']/tbody/tr[{row}]/td[2]".format(row=row))[0].text.replace('：', '')

            prod_info[key1] = val1
            prod_info[key2] = val2

            row += 1
            
        row2 = 1
        while row2<=3:
            key3 = tree2.xpath("//div[@class='maincont']/div[@class='mainbox']/div[@class='mbox']/div[@class='dzMain']/div[@class='dzMain'][4]/div/table[@class='dzTable']/tbody/tr[{row}]/th[1]".format(row=row2))[0].text.replace('：', '')
            val3 = tree2.xpath("//div[@class='maincont']/div[@class='mainbox']/div[@class='mbox']/div[@class='dzMain']/div[@class='dzMain'][4]/div/table[@class='dzTable']/tbody/tr[{row}]/td[1]".format(row=row2))[0].text.replace('：', '')
            key4 = tree2.xpath("//div[@class='maincont']/div[@class='mainbox']/div[@class='mbox']/div[@class='dzMain']/div[@class='dzMain'][4]/div/table[@class='dzTable']/tbody/tr[{row}]/th[2]".format(row=row2))[0].text.replace('：', '')
            val4 = tree2.xpath("//div[@class='maincont']/div[@class='mainbox']/div[@class='mbox']/div[@class='dzMain']/div[@class='dzMain'][4]/div/table[@class='dzTable']/tbody/tr[{row}]/td[2]".format(row=row2))[0].text.replace('：', '')

            prod_info[key3] = val3
            prod_info[key4] = val4

            row2 += 1
        
        row3 = 4
        while row3<=9:
            key5 = tree2.xpath("//div[@class='maincont']/div[@class='mainbox']/div[@class='mbox']/div[@class='dzMain']/div[@class='dzMain'][4]/div/table[@class='dzTable']/tbody/tr[{row}]/th".format(row=row3))[0].text.replace('：', '')
            val5 = tree2.xpath("//div[@class='maincont']/div[@class='mainbox']/div[@class='mbox']/div[@class='dzMain']/div[@class='dzMain'][4]/div/table[@class='dzTable']/tbody/tr[{row}]/td".format(row=row3))[0].text.replace('：', '')
             
            prod_info[key5] = val5
            
            row3 += 1
            
        all_info.append(prod_info)

        i += 1
        
    p += 1

with open('/Users/Tianyu/Documents/OneConnect/Hackathon/Templates5/bank-manager-tool/backend/data/hx_dep_prd2.json', 'a', encoding="utf-8") as f:
    json.dump(all_info, f, ensure_ascii=False, indent=4)
    f.write('\n')

搏弈BYAQKF280D
日日聚财(1401期)A19003
日日聚财(1401期)A19002
日日聚金1501期G26008
红棉理财C1082720A000010
紫气东来稳健1294期
翠竹同益3M对公10款(B)19期FGAB15044B
翠竹同益3M对公10款(A)19期FGAB15044A
紫气东来稳健1293期
翠竹同益1M54期FGAB15016A
翠竹同益3M对公09款(B)18期FGAB15042B
翠竹同益3M对公09款(A)18期FGAB15042A
成长2020年第74期
安稳2020年第42期
翠竹同益3M对公08款(B)18期FGAB15040B
翠竹同益3M对公08款(A)18期FGAB15040A
成长2020年第73期
稳健成长(2020)86期
稳健成长(2020)88期
合赢2020年第4期
成长2020年第71期
成长2020年第72期
金翼宝942号
安盈268号
安盈269号
安盈270号
安盈271号
安盈272号
安盈273号
安盈274号
安盈275号
安盈276号
安盈277号
安盈278号
安盈279号
安盈280号
稳健成长(2020)85期
金翼宝941号
稳健成长(2020)84期
翠竹5W周四尊享02款FSAB13004D
翠竹5W周四高端02款FSAB13004B
翠竹5W周四普通02款FSAB13004A
稳健成长(2020)83期
稳健成长(2020)82期
紫气东来稳健1292期
工银财富CQXT2020
金翼宝940号
漓江理财2020-52期
成长2020年第70期
安稳2020年第41期
翠竹同益3M对公07款(B)18期FGAB15038B
翠竹同益3M对公07款(A)18期FGAB15038A
中银平稳智荟208496期
稳健成长(2020)79期
安稳2020年第40期
创赢计划青鑫共享2020年186期(封闭)
创赢计划青鑫共享2020年185期(封闭)
创赢计划青鑫共享2020年174期(封闭)
创赢计划青鑫共享2020年173期(封闭)
金翼宝939号
稳健成长(2020)81期
2020年长吉39期2020089
蒙银财鑫20057期
蒙银财鑫20056期
农银私行·安心得利·灵珑AD200210
优选恒利1号2020年第19期
金紫薇共赢43号封闭净值
金紫薇共赢42号封

结构性存款JGP20083
本利丰BFDG200028
2020年私募122号
升盈330号B款
卓越增盈第200137期预约46天
卓越增盈第200136期预约35天
结构性存款产品JGP20081
汇利丰HF200721
汇利丰HF200720
蒙银财慧20053期A款
蒙银财慧20053期B款
蒙银财慧20053期C款
汇利丰HF200705
汇利丰HF200704
汇利丰HF200703
焦点联动118088
汇利丰HF200702
汇利丰HF200701
汇利丰HF200700
富利宝封闭式民币(FJ20009期)
焦点联动119868
焦点联动118085
如意添财第2029期
得利宝·私银慧享6个月2463200085
得利宝·私银慧享2463200084
得利宝·私银慧享2463200083
汇利丰HF200693
汇利丰HF200692
汇利丰HF200691
汇利丰HF200690
汇利丰HF200689
汇利丰HF200688
汇利丰HF200687
益民盈鑫576号
龙盈固收TS款15号
中银平稳智荟208606期
中银平稳智荟208611期
天天万汇通92120018
天天万汇通92120017
海融财富钱潮2020年63期
海融财富钱潮2020年64期
海融财富钱潮2020年62期
小龙人和盛灵龙255号
小龙人和盛龙瑞891号
旺息宝1号20055期封闭式(市民卡专属)
旺息宝1号20056期封闭式
旺息宝1号20057期封闭式
旺息宝2号20019期封闭式
稳金20019期封闭式
金钥匙·安心得利·灵珑第208期
得利宝·交银添利2142200073
映山红安享回报145号
金钥匙·安心得利·灵珑第203期
交银添利6个月深2142200072
得利宝·交银添利2163200091
得利宝·交银添利2163200090
沃德添利3个月深2142200070
乾元-豫富2020年第20期
臻选鼎鼎A款34号
臻选鼎鼎A款33号
B计划(美元)2020年第19期
A计划(美元)2020年第86期
A计划(美元)2020年第85期
邮银财富瑞享2020年第63期
邮银财富债券2020年第68期
邮银财富智享2020年第36期
交银添利特邀沪C1030120000754
浙江省分行乾元赢来赢往2020年第3期封闭式
慧盈971号D款20104

In [17]:
with open('/Users/Tianyu/Documents/OneConnect/Hackathon/Templates5/bank-manager-tool/backend/data/hx_dep_prd2.json', encoding='utf-8') as data_file:
    data = json.loads(data_file.read())

In [18]:
df = pd.DataFrame(data)

In [19]:
df['发行银行'].value_counts()

农业银行    67
民生银行    41
建设银行    37
中国银行    29
交通银行    28
广州农商    26
兴业银行    24
工商银行    23
杭州银行    23
华夏银行    18
晋商银行    16
吉林银行    15
青岛银行    15
招商银行    14
稠州银行    14
宁波银行    13
绍兴银行    12
珠海华润    11
上海银行    11
邮储银行    11
广州银行    11
大连银行    10
上海农商    10
内蒙古银    10
威海市商    10
桂林银行     9
中信银行     9
长沙银行     9
兰州银行     7
南昌银行     7
九江银行     6
营口银行     6
杭州联合     6
长春农商     6
南商中国     6
成都银行     6
齐商银行     5
乐山商行     5
包商银行     5
天津银行     5
天津农商     5
金华银行     5
广东华兴     5
盛京银行     5
广东南粤     5
郑州银行     5
新韩银行     5
湖北银行     4
上饶银行     4
锦州银行     4
泰安市商     4
三峡银行     3
乌鲁木齐     3
龙江银行     3
南京银行     3
鄞州农村     3
广西北部     3
成都农商     3
哈尔滨银     3
洛阳银行     3
滨海农商     3
河北银行     3
浙江民泰     2
西安银行     2
北京银行     2
恒丰银行     2
嘉兴银行     2
齐鲁银行     2
厦门银行     2
平安银行     2
重庆农商     2
慈溪农村     2
紫金农商     2
德阳银行     2
华融湘江     2
浦发银行     2
海峡银行     2
汉口银行     1
日照银行     1
太仓农村     1
攀枝花市     1
江苏银行     1
南充市商     1
光大银行     1
兴化农村     1
潍坊银行     1
星展银行     1
Name: 发行银行, dtype: int64

In [20]:
df.columns

Index(['产品名称', '图片地址', '发行银行', '委托币种', '投资类型', '委托币种起始金额', '预期收益率(%)',
       '起始金额递增单位', '到期收益率(%)', '同期存款利率比较', '是否保本', '发行起始日期', '可否质押', '发行终止日期',
       '委托管理期(月)', '收益起始日期', '付息周期(月)', '收益终止日期', '收益类型', '委托附属币种',
       '客户是否有权提前赎回', '附属币种起始金额', '银行是否有权提前终止', '附属币种起始金额递增单位', '适用地区', '产品管理费',
       '申购赎回规定', '投资对象', '收益率说明', '投资风险说明'],
      dtype='object')

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 30 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   产品名称          720 non-null    object
 1   图片地址          720 non-null    object
 2   发行银行          720 non-null    object
 3   委托币种          720 non-null    object
 4   投资类型          720 non-null    object
 5   委托币种起始金额      720 non-null    object
 6   预期收益率(%)      720 non-null    object
 7   起始金额递增单位      720 non-null    object
 8   到期收益率(%)      720 non-null    object
 9   同期存款利率比较      720 non-null    object
 10  是否保本          720 non-null    object
 11  发行起始日期        720 non-null    object
 12  可否质押          720 non-null    object
 13  发行终止日期        720 non-null    object
 14  委托管理期(月)      720 non-null    object
 15  收益起始日期        720 non-null    object
 16  付息周期(月)       720 non-null    object
 17  收益终止日期        720 non-null    object
 18  收益类型          720 non-null    object
 19  委托附属币种  

In [22]:
df['预期收益率(%)'].value_counts()

--      334
3.65     23
4.00     22
0.00     18
3.85     15
4.15     13
3.50     12
3.90     11
4.20      9
4.10      9
3.60      9
3.75      9
3.80      9
3.55      9
5.65      8
4.50      8
3.70      7
4.05      6
3.95      6
5.05      6
3.45      5
4.65      5
6.00      5
5.00      5
5.45      5
4.80      5
5.50      5
4.60      5
4.70      4
4.75      4
4.40      4
4.02      4
4.35      4
4.30      4
3.35      4
3.40      4
4.25      4
4.45      3
3.25      3
2.62      3
3.00      3
3.48      3
5.90      3
4.55      3
3.30      3
7.00      2
9.00      2
3.63      2
5.75      2
5.95      2
3.20      2
2.80      2
5.03      2
6.05      2
6.65      2
1.30      2
5.25      2
5.20      2
1.20      1
2.65      1
7.90      1
1.70      1
1.45      1
1.35      1
4.95      1
2.95      1
1.80      1
9.60      1
6.50      1
4.09      1
7.10      1
3.46      1
4.33      1
4.08      1
2.85      1
6.95      1
9.20      1
3.37      1
1.92      1
4.59      1
8.00      1
2.90      1
3.05      1
1.55

In [23]:
df['委托管理期(月)'].value_counts()

3.03      64
3.07      29
6.10      25
1.17      25
3.27      25
12.17     24
3.00      22
6.07      22
6.30      17
6.00      17
4.20      13
6.27      11
2.10      10
5.83       9
3.10       9
12.13      9
6.23       9
3.20       8
6.33       8
1.20       8
0.00       8
3.13       8
7.00       7
5.10       7
6.03       7
5.13       7
3.23       6
3.73       6
6.17       6
3.50       6
5.80       6
3.97       6
6.97       5
3.17       5
3.30       5
5.03       4
1.13       4
3.47       4
6.47       4
2.97       4
24.20      3
4.13       3
12.30      3
3.53       3
4.90       3
121.73     3
7.13       3
3.93       3
1.10       3
7.03       3
5.17       3
4.17       3
9.57       3
6.67       3
4.47       2
2.03       2
6.53       2
13.23      2
6.37       2
6.20       2
1.93       2
1.97       2
5.87       2
5.07       2
3.37       2
4.67       2
5.60       2
7.23       2
11.80      2
1.00       2
5.57       2
5.40       2
7.40       2
1.27       2
24.93      2
6.70       2
12.07      2

In [24]:
df.shape

(720, 30)